# Simple Wave and Wind Hybrid Plant Example
---
In this example, we will walk through the process of simulating a hybrid renewable energy system including both wave and wind energy.

### Import Required Modules
We start by importing the necessary modules and setting up our working environment.

In [1]:
from hopp.simulation.technologies.sites import SiteInfo
from hopp.simulation.technologies.sites import flatirons_site as sample_site
from hopp.simulation.hybrid_simulation import HybridSimulation
from hopp.utilities.keys import set_nrel_key_dot_env

/Users/kbrunik/github/forked/HOPP/examples/workshop/log/hybrid_systems_2023-09-28T09.17.44.626966.log


### Set API Key
To access the Wind Toolkit (wind resource) and NSRDB (solar resource) data, we need to set an API key. You can obtain an API key from the [NREL developer website](https://developer.nrel.gov/signup/).

To set up the `NREL_API_KEY` required for resource downloads, you can create an Environment Variable called `NREL_API_KEY`. Otherwise, you can keep the key in a new file called ".env" in the root directory of this project.

Create a file ".env" that contains the single line:
`NREL_API_KEY=key`

In [2]:
# Set API key using the .env
set_nrel_key_dot_env()

#### (Optional)

This step is if you do not set the API key using a ".env" file.

In [ ]:
import os
from hopp.utilities.keys import set_developer_nrel_gov_key

# Set API key manually if not using the .env
global NREL_API_KEY
NREL_API_KEY = os.getenv("NREL_API_KEY") # Set this key manually here if you are not setting it using the .env
set_developer_nrel_gov_key(NREL_API_KEY)  

### Retrieve Resource Data
Obtain the resource data for your location. Set the `site_input_data` dictionary with the desired location, year, and resources used in hybrid plant.

The Wind resource data will be downloaded using the API key. The Wave resource data must be pre-loaded in the format given in the `Wave_resource_timeseries.csv`.

In [9]:
from hopp import ROOT_DIR

site_input_data = sample_site
site_input_data['lat'] = 44.6899
site_input_data['lon'] = -124.1346
site_input_data['year'] = 2010

DEFAULT_WAVE_RESOURCE_FILE = ROOT_DIR.parent / "resource_files" / "wave" / "Wave_resource_timeseries.csv"

site = SiteInfo(
    site_input_data,
    wave_resource_file=DEFAULT_WAVE_RESOURCE_FILE,
    solar=False, 
    wind=True, 
    wave=True
)

### Read in Wave Technology Configuration
The Wave configuration is defined in a `yaml` file that has the wave power matrix and device information.

In [15]:
import yaml

mhk_yaml_path = "inputs/wave/wave_device.yaml"
with open(mhk_yaml_path, 'r') as stream:
    mhk_config = yaml.safe_load(stream)



### Setup Financial Model Paremeters

The wave model does not have a default financial model so we have to set up the `CustomFinancialModel` from HOPP.

The `default_fin_config` contains all of the necessary parameters for the financials to be calculated when running `HybridSimulation`.

In [16]:
from hopp.simulation.technologies.financial.custom_financial_model import CustomFinancialModel

default_fin_config = {
	'batt_replacement_schedule_percent': [0],
	'batt_bank_replacement': [0],
	'batt_replacement_option': 0,
	'batt_computed_bank_capacity': 0,
	'batt_meter_position': 0,
	'om_fixed': [1],
	'om_production': [2],
	'om_capacity': (0,),
	'om_batt_fixed_cost': 0,
	'om_batt_variable_cost': [0],
	'om_batt_capacity_cost': 0,
	'om_batt_replacement_cost': 0,
	'om_replacement_cost_escal': 0,
	'system_use_lifetime_output': 0,
	'inflation_rate': 2.5,
	'real_discount_rate': 6.4,
	'cp_capacity_credit_percent': [0],
	'degradation': [0],
	'ppa_price_input': (0.01,),
	'ppa_escalation': 1
	}

### Define Technologies Configuration
Configure the characteristics of each technology in the system, including wind, wave, and grid interconnection.

The technologies all have to use the same financial model so you can see that we call the `CustomFinancialModel` for each technology.

In [17]:
interconnection_size_kw = 15000
wind_kw = 10000

technologies = {'wind': {
                    'num_turbines': 5,
                    'turbine_rating_kw': wind_kw / 5,
                    'fin_model': CustomFinancialModel(default_fin_config)
                },
                'wave':{
                    'device_rating_kw': mhk_config['device_rating_kw'], 
                    'num_devices': 10, 
                    'wave_power_matrix': mhk_config['wave_power_matrix'],
                    'fin_model': CustomFinancialModel(default_fin_config)
                    },
                'grid': {
                    'interconnect_kw': interconnection_size_kw,
                    'fin_model': CustomFinancialModel(default_fin_config)
                }}

### Create the Hybrid Simulation Model
Create an instance of the `HybridSimulation` class by providing the technology configuration and site information.

In [18]:
hybrid_plant = HybridSimulation(technologies, site)

### Add Wave Cost Model Inputs
Add the inputs to run the wave technology cost model and instantiate `MHKCosts`.

In [19]:
cost_model_inputs = {
	'reference_model_num':3,
	'water_depth': 100,
	'distance_to_shore': 80,
	'number_rows': 10,
	'device_spacing':600,
	'row_spacing': 600,
	'cable_system_overbuild': 20
	}

hybrid_plant.wave.create_mhk_cost_calculator(cost_model_inputs)

### Run the Simulation
Simulate the hybrid renewable energy system for a specified number of years (in this case, 25 years).

In [20]:
hybrid_plant.simulate(project_life = 25)

### Retrieve and Display Results
Access the simulation results, including annual energies, capacity factors and net present values (NPVs), and print them to the console.

In [21]:
aeps = hybrid_plant.annual_energies
npvs = hybrid_plant.net_present_values
cf = hybrid_plant.capacity_factors

print("Annual Energy Production")
print(aeps)
print("Net Present Value")
print(npvs)
print("Capacity Factors")
print(cf)

Annual Energy Production
{"wind": 32539658.835298087, "wave": 12132526.0, "hybrid": 44672184.83529807}
Net Present Value
{"wind": -14977200.729403041, "wave": -53731805.52113224, "hybrid": -68709006.25053555}
Capacity Factors
{"wind": 37.14572926403891, "wave": 48.426278059839696, "hybrid": 39.654467176635364}
